### Fine-tuning ChatGPT (GPT-3.5-Turbo)

In [53]:
# Load Packages
import json
import random
import pandas as pd
from tqdm import tqdm

#### 0. Cilent of OpenAI

In [ ]:
# !pip install -U openai
from openai import OpenAI
client = OpenAI(api_key='Your OpenAI API-KEY')

#### 1. Prepare Data (where different for different tasks)

In [ ]:
# process raw data into .csv
data_splits = ['train', 'train-augmented-unique', 'valid', 'test']

for data_split in data_splits:
    
    with open(f"hand_annotated/src-{data_split}.txt", "r", encoding='utf-8') as f:
        paragraphs = f.readlines()
        
    with open(f"hand_annotated/tgt-{data_split}.txt", "r", encoding='utf-8') as f:
        actions = f.readlines()
        
    paragraphs = [_.rstrip("\n") for _ in paragraphs]
    actions = [_.rstrip("\n") for _ in actions]

    valid_df = pd.DataFrame()
    valid_df['paragraphs'] = paragraphs
    valid_df['actions'] = actions
    valid_df.to_csv(f"processed/hand_annotated_{data_split}.csv", index = None)

In [2]:
# process .csv into .jsonl
def transform_jsonl(input_file_path, output_file_path):
    train_df = pd.read_csv(input_file_path)
    
    with open(output_file_path, 'w') as outfile:
        for i in range(len(train_df)):
            messages = []
            messages.append({"role": "system", "content": "You are an assistant that formats chemical paragraphs to actions."})
            user_message = {"role": "user", "content": train_df.iloc[i]['paragraphs']}
            assistant_message = {"role": "assistant", "content": train_df.iloc[i]['actions']}
            messages.extend([user_message, assistant_message])
            result = {"messages": messages}
            json.dump(result, outfile)
            outfile.write('\n')

input_file_path = "processed/hand_annotated_train_augmented_unique.csv"
output_file_path = 'processed/hand_annotated_train_augmented_unique_ouput.jsonl'
transform_jsonl(input_file_path, output_file_path)

input_file_path = "processed/hand_annotated_valid.csv"                           
output_file_path = 'processed/hand_annotated_valid_ouput.jsonl'
transform_jsonl(input_file_path, output_file_path)

#### 2. Upload Data

In [ ]:
# Wait a few minutes to process and validate the data, After uploading files

training_response = client.files.create(
    file = open('processed/hand_annotated_train_augmented_unique_ouput.jsonl', "rb"),
    purpose = "fine-tune",
)

validation_response = client.files.create(
    file = open('processed/hand_annotated_valid_ouput.jsonl', "rb"),
    purpose = "fine-tune",
)

print("Training file:", training_response.model_dump_json(indent=2))
print("Validating file:", validation_response.model_dump_json(indent=2))

training_file_id = training_response.id
validation_file_id = validation_response.id

#### 3. Create a fine-tunning job

In [6]:
fine_tuning_response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-3.5-turbo-0125",         # newest gpt-3.5-turbo model
    suffix = "paragraph2action",
    hyperparameters = {"n_epochs":3,}
)

print('fine_tuning_response:', fine_tuning_response.model_dump_json(indent=2))
fine_tuning_job_id = fine_tuning_response.id 

{
  "object": "fine_tuning.job",
  "id": "ftjob-2owUkGkhZI8ijvpzUG3c85MI",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695785269,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-xKjEtEsxu2H7NAxjpmNLOanI",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-LE74xWWtMRyNcmx7nda51Vu8",
  "training_file": "file-Sb2ZG8nRhswCnWGvZYofJ9rI",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}


#### 4. Query the progress of Fine-tuning

In [7]:
# run the cell to see the status is "running" or "succeeded"

retrieve_job_response = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
print(retrieve_job_response.model_dump_json(indent=2))
print('status:', retrieve_job_response.status)
fine_tuned_model = retrieve_job_response.fine_tuned_model       # get the fine-tuned model

<FineTuningJob fine_tuning.job id=ftjob-2owUkGkhZI8ijvpzUG3c85MI at 0x22b70515760> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-2owUkGkhZI8ijvpzUG3c85MI",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695785269,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-xKjEtEsxu2H7NAxjpmNLOanI",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-LE74xWWtMRyNcmx7nda51Vu8",
  "training_file": "file-Sb2ZG8nRhswCnWGvZYofJ9rI",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

In [50]:
# run the cell frequently to see the training loss

for event in client.fine_tuning.jobs.list_events(fine_tuning_job_id, limit = 50):
    print(event.message)

Created fine-tuning job: ftjob-2owUkGkhZI8ijvpzUG3c85MI
Validating training file: file-Sb2ZG8nRhswCnWGvZYofJ9rI and validation file: {ft_job.validation_file}
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1518: training loss=0.82, validation loss=1.28
Step 101/1518: training loss=0.06, validation loss=0.13
Step 201/1518: training loss=0.01, validation loss=0.09
Step 301/1518: training loss=0.02, validation loss=0.05
Step 401/1518: training loss=0.01, validation loss=0.18
Step 501/1518: training loss=0.02, validation loss=0.05
Step 601/1518: training loss=0.07, validation loss=0.05
Step 701/1518: training loss=0.04, validation loss=0.07
Step 801/1518: training loss=0.00, validation loss=0.06
Step 901/1518: training loss=0.00, validation loss=0.06
Step 1001/1518: training loss=0.04, validation loss=0.07
Step 1101/1518: training loss=0.02, validation loss=0.12
Step 1201/1518: training loss=0.00, validation loss=0.06
Step 1301/1518: training loss=0.03, validatio

#### 5. Inference with the fine-tuned model

In [52]:
# Single Example

test_messages = []
syst_message = "You are an assistant that formats chemical paragraphs to actions."
user_message = "After then being left to stand at RT for three days, the mixture was diluted with ethyl acetate and washed with water."
test_messages.append({"role":"system", "content": syst_message})
test_messages.append({"role":"user", "content":user_message})

response = client.chat.completions.create(
  model = fine_tuned_model,
  messages = test_messages,
  temperature = 0,
  # max_tokens = 500
)

print(response)
print("Paragraphs: ", user_message)
print("Actions: ", response.choices[0].message.content)

Paragraphs:  After then being left to stand at RT for three days, the mixture was diluted with ethyl acetate and washed with water.
{
  "id": "chatcmpl-83H3QGurgbnFI967FKxt21IIOekuM",
  "object": "chat.completion",
  "created": 1695791232,
  "model": "ft:gpt-3.5-turbo-0613:personal:paragraph2action:83GrVYSz",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "WAIT for three days at RT; ADD ethyl acetate; WASH with water."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 48,
    "completion_tokens": 18,
    "total_tokens": 66
  }
}
Actions:  WAIT for three days at RT; ADD ethyl acetate; WASH with water.


In [55]:
## Whole test
test_df = pd.read_csv("processed/hand_annotated_test.csv")
test_paragraphs = list(test_df['paragraphs'])
test_actions_preds = []

for user_message in tqdm(test_paragraphs):
  test_messages = []
  syst_message = "You are an assistant that formats chemical paragraphs to actions."
  # user_message = "After then being left to stand at RT for three days, the mixture was diluted with ethyl acetate and washed with water."
  test_messages.append({"role":"system", "content": syst_message})
  test_messages.append({"role":"user", "content": user_message})

  response = client.chat.completions.create(
    model = fine_tuned_model,
    messages = test_messages,
    temperature = 0,
    # max_tokens = 500
  )
  # print(response)

  pred = response["choices"][0]["message"]["content"]
  print("\nParagraphs: ", user_message)
  print("Actions: ", pred)
  test_actions_preds.append(pred)

  0%|          | 0/352 [00:00<?, ?it/s]


Paragraphs:  Both the organic and aqueous layers were separated.


  0%|          | 1/352 [00:15<1:29:47, 15.35s/it]

Actions:  PHASESEPARATION.

Paragraphs:  After then being left to stand at RT for three days, the mixture was diluted with ethyl acetate and washed with water.


  1%|          | 2/352 [00:17<43:45,  7.50s/it]  

...

Actions:  ADD mixture of ice-water (150 ml) and sodium hydrogencarbonate (10 g); EXTRACT with ether (200 ml) 2 x; YIELD.

Paragraphs:  The crystallographic data is summarized in the table below.


100%|█████████▉| 351/352 [04:53<00:00,  1.23it/s]

Actions:  NOACTION.

Paragraphs:  First, precursor H-ValCH₂Cl·HCl was prepared by dissolving Boc-Val-OH (6.5 g, 30 mmol) in 10 mL of THF and treating it with N-methylmorpholine (3.3 mL, 30 mmol) and isobutyl chloroformate (3.9 mL, 30 mmol) for 10 minutes at -20°.


100%|██████████| 352/352 [04:53<00:00,  1.20it/s]

Actions:  INVALIDACTION.


### 6. Save Predictions

In [57]:
actuals = list(pd.read_csv("processed/hand_annotated_test.csv")['actions'])
final_df = pd.DataFrame({"Generated Text": test_actions_preds, "Actual Text": actuals})
final_df.to_csv(f"results/output_gpt3.5_hand_annotated_train_augmented_unique_3_epoch.csv")